In [1]:
import numpy as np
import pandas as pd
import seaborn

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor

from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cont_features = ["cont"+str(i+1) for i in range(14)]
cat_features = ["cat"+str(i+1) for i in range(116)]
target = ["loss"]

#unique_categories = {c:np.unique(list(train_data[c])+list(test_data[c])) for c in cat_features}
unique_categories = {x[0]:x[1].split() for x in pd.read_csv("categories.csv").as_matrix()}

In [3]:
#encoder
def encoder(df, cat_features, cat_dict):
    tr_data = df.copy()

    new_cat_features = []

    for col in cat_features:
        uniq = cat_dict[col]
        if len(uniq) == 2:
            tr_data[col+'_0'] = np.array(map(int, tr_data[col]==uniq[0]))
            new_cat_features += [col+'_0']
        else:
            for j, v in enumerate(uniq):
                tr_data[col+'_'+str(j)] = np.array(map(int, tr_data[col]==uniq[j]))
                new_cat_features += [col+'_'+str(j)]
    return tr_data.copy(), new_cat_features

In [4]:
reader = pd.read_csv("train.csv", chunksize=2500)

models = []
step = 0

for train_data in reader:
    
    step += 1
    
    tr_data, new_cat_features = encoder(train_data, cat_features, unique_categories)
    #tr_data = tr_data.reindex(np.random.permutation(tr_data.index))
    #train_data, val_data = tr_data[:1000], tr_data[1000:2000]

    X_train, y_train = tr_data[cont_features+new_cat_features].as_matrix(), tr_data[target].as_matrix().transpose()[0]
    #X_val, y_val = val_data[cont_features+new_cat_features].as_matrix(), val_data[target].as_matrix().transpose()[0]
    
    model = RandomForestRegressor(n_estimators=1, criterion="mae", max_depth=3)
    
    model.fit(X_train, y_train)
    models += [model]
    print "step ",step," done"
    

step  1  done
step  2  done
step  3  done
step  4  done
step  5  done
step  6  done
step  7  done
step  8  done
step  9  done
step  10  done
step  11  done
step  12  done
step  13  done
step  14  done
step  15  done
step  16  done
step  17  done
step  18  done
step  19  done
step  20  done
step  21  done
step  22  done
step  23  done
step  24  done
step  25  done
step  26  done
step  27  done
step  28  done
step  29  done
step  30  done
step  31  done
step  32  done
step  33  done
step  34  done
step  35  done
step  36  done
step  37  done
step  38  done
step  39  done
step  40  done
step  41  done
step  42  done
step  43  done
step  44  done
step  45  done
step  46  done
step  47  done
step  48  done
step  49  done
step  50  done
step  51  done
step  52  done
step  53  done
step  54  done
step  55  done
step  56  done
step  57  done
step  58  done
step  59  done
step  60  done
step  61  done
step  62  done
step  63  done
step  64  done
step  65  done
step  66  done
step  67  done
step

In [5]:
train_data = pd.read_csv("train.csv")
train_data, ncf = encoder(train_data, cat_features, unique_categories)

X_train = train_data[cont_features+ncf].as_matrix()
y_train = train_data[target].as_matrix().transpose()[0]

pred = np.zeros(train_data.shape[0])

for step, model in enumerate(models):
    pred += model.predict(X_train)
    print "step ",step+1," done"
pred = pred / float(len(models))

pred = np.array(map(lambda x: max(x,0.0), pred))
print np.mean(np.abs(pred - train_data[target].as_matrix().transpose()[0]))
#answer = pd.DataFrame(zip(test_data["id"], pred), columns=["id", "loss"])
#answer.to_csv("submission.csv", index=False)

step  1  done
step  2  done
step  3  done
step  4  done
step  5  done
step  6  done
step  7  done
step  8  done
step  9  done
step  10  done
step  11  done
step  12  done
step  13  done
step  14  done
step  15  done
step  16  done
step  17  done
step  18  done
step  19  done
step  20  done
step  21  done
step  22  done
step  23  done
step  24  done
step  25  done
step  26  done
step  27  done
step  28  done
step  29  done
step  30  done
step  31  done
step  32  done
step  33  done
step  34  done
step  35  done
step  36  done
step  37  done
step  38  done
step  39  done
step  40  done
step  41  done
step  42  done
step  43  done
step  44  done
step  45  done
step  46  done
step  47  done
step  48  done
step  49  done
step  50  done
step  51  done
step  52  done
step  53  done
step  54  done
step  55  done
step  56  done
step  57  done
step  58  done
step  59  done
step  60  done
step  61  done
step  62  done
step  63  done
step  64  done
step  65  done
step  66  done
step  67  done
step

In [ ]:
test_data = pd.read_csv("test.csv")
test_data, ncf = encoder(test_data, cat_features, unique_categories)

X_test = test_data[cont_features+ncf].as_matrix()
pred = np.zeros(test_data.shape[0])

for model in models:
    pred += model.predict(X_test)
pred = pred / float(len(models))

pred = np.array(map(lambda x: max(x,0.0), pred))
answer = pd.DataFrame(zip(test_data["id"], pred), columns=["id", "loss"])
answer.to_csv("submission.csv", index=False)

In [ ]:
cvs_train, cvs_val = [], []
rang = np.arange(1,11,1)

for n in rang:
    model = BaggingRegressor(
        RandomForestRegressor(criterion='mae', n_estimators=10, max_features="log2"), 
        n_estimators=n)
    
    model.fit(X_train, y_train)
    pred = model.predict(X_train)
    cvs_train += [np.mean(np.abs(y_train-pred))]
    
    pred = model.predict(X_val)
    cvs_val += [np.mean(np.abs(y_val-pred))]
    
    #cvs += [np.mean(cross_val_score(model, X, y, scoring="neg_mean_absolute_error"))]
    print n

#rang = range(1,122,5)
plt.plot(rang, cvs_train, rang, cvs_val)

In [ ]:
cvs_train, cvs_val = [], []
#tr1_data = tr_data.copy()
rang = range(10)

for i in rang:
    tr1_data = tr1_data.reindex(np.random.permutation(tr1_data.index))
    train_data, val_data = tr1_data[:1000], tr1_data[1000:2000]
    X_train, y_train = train_data[cont_features+new_cat_features].as_matrix(), func(train_data[target].as_matrix().transpose()[0])
    X_val, y_val = val_data[cont_features+new_cat_features].as_matrix(), func(val_data[target].as_matrix().transpose()[0])
    
    #model = BaggingRegressor(RandomForestRegressor(criterion='mae', n_estimators=10, max_features="log2"), n_estimators=10)
    model = GradientBoostingRegressor(loss='lad', learning_rate=0.3, n_estimators=100)
    
    model.fit(X_train, y_train)
    pred = invfunc(model.predict(X_train))
    cvs_train += [np.mean(np.abs(y_train-pred))]
    
    pred = invfunc(model.predict(X_val))
    cvs_val += [np.mean(np.abs(y_val-pred))]
    
    print i
    
plt.plot(rang, cvs_train, rang, cvs_val)

In [ ]:
model

In [ ]:
test_data = pd.read_csv("test.csv")
test_data, ncf = encoder(test_data, cat_features, unique_categories)

In [ ]:
#test_data = pd.read_csv("test.csv")

X_test = test_data[cont_features+ncf].as_matrix()
pred = model.predict(X_test)
pred = np.array(map(lambda x: max(x,0.0), pred))
answer = pd.DataFrame(zip(test_data["id"], pred), columns=["id", "loss"])
answer.to_csv("submission.csv", index=False)

In [9]:
modelX_train = np.array(map(lambda x: x.predict(X_train), models)).transpose()

In [28]:
finalmodel = GradientBoostingRegressor(loss='lad', max_depth=11, max_features="log2", n_estimators=111)
finalmodel.fit(modelX_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=11,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=111, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [11]:
pred = finalmodel.predict(modelX_train)
print np.mean(np.abs(pred - y_train))

In [12]:
print -np.mean(cross_val_score(
                BaggingRegressor(
                GradientBoostingRegressor(loss='lad', max_depth=3, max_features="log2", n_estimators=100), 
                n_estimators=10), 
                          modelX_train, y_train, 
                          scoring="neg_mean_absolute_error", cv=3))

1303.8290255


In [30]:
test_data = pd.read_csv("test.csv")
test_data, ncf = encoder(test_data, cat_features, unique_categories)

X_test = test_data[cont_features+ncf].as_matrix()
modelX_test = np.array(map(lambda x: x.predict(X_test), models)).transpose()
pred = finalmodel.predict(modelX_test)
pred = np.array(map(lambda x: max(x,0.0), pred))
answer = pd.DataFrame(zip(test_data["id"], pred), columns=["id", "loss"])
answer.to_csv("submission.csv", index=False)